In [1]:

####################
#Author: brandon chiazza
#version: 1.0
#purpose: to call a twitter api and return results
#documentation: https://developer.twitter.com/en/docs
#####################

import pandas as pd
import requests
import json
import base64
import s3fs # documentation: https://s3fs.readthedocs.io/en/latest/
import time
import twitter_keys #this is a custom reference module to a package containing twitter keys

%config IPCompleter.greedy=True


key_secret = '{}:{}'.format(twitter_keys.client_key, twitter_keys.client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

#identify base url and oauth token path
base_url = 'https://api.twitter.com/' #base url for authentication
auth_url = '{}oauth2/token'.format(base_url)

#share header information -- encoding is ascii
auth_headers = {
    'Authorization': 'Basic {}'.format(b64_encoded_key),
    'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'
}

#pass clientcredentials
auth_data = {
    'grant_type': 'client_credentials'
}

#send authentication using requests - POST request
auth_resp = requests.post(auth_url, headers=auth_headers, data=auth_data)

#check response status. 200 = OK
auth_resp.status_code




200

In [2]:

#Keys in data response are token_type (bearer) and access_token (your access token)
print(auth_resp.json().keys())

access_token = auth_resp.json()['access_token']


search_headers = {
    'Authorization': 'Bearer {}'.format(access_token)    
}

#enter search parameters for coronavirus example. This looks for "covid-19" in the 1000 most recent tweets
query_params = {
    'q': 'Olympic',
    'result_type': 'recent',
    'count': 1000, #update here to get more/less than 1000 returns
    'lang': 'en' #filters by english language only
}


#identify search url path and save 
search_url = '{}1.1/search/tweets.json'.format(base_url)


#run search using get request
search_resp = requests.get(search_url, headers=search_headers, params=query_params)

#check status code of GET request
search_resp.status_code


dict_keys(['token_type', 'access_token'])


200

In [3]:
#print text from result to verify  
twitter_data = search_resp.json()

for x in twitter_data['statuses']:
    print(x['text'] + '\n')
    break #prints after one iteration and stops, remove break to see all 1000

RT @NBCOlympics: "This should be an Olympic sport."

41 Olympic sports, 50 disciplines as their pictograms - in under five minutes.

#Tokyo…



In [4]:
# move data into data frame 
df = pd.DataFrame(twitter_data['statuses'])

# show one record to verify import 
df.head(20)

,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,...,retweet_count,favorite_count,favorited,retweeted,lang,quoted_status_id,quoted_status_id_str,extended_entities,possibly_sensitive,quoted_status
0,Sat Jul 24 22:54:51 +0000 2021,1419068590855278594,1419068590855278594,"RT @NBCOlympics: ""This should be an Olympic sp...",False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,369,0,False,False,en,NaN,NaN,NaN,NaN,NaN
1,Sat Jul 24 22:54:51 +0000 2021,1419068589525643267,1419068589525643267,RT @Smoore1117: The absolute best summer Olymp...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,1,0,False,False,en,1.418724e+18,1418723522927644675,NaN,NaN,NaN
2,Sat Jul 24 22:54:49 +0000 2021,1419068581158006784,1419068581158006784,RT @busenazckroglu: Olympic opening ceremony🤍 ...,False,"{'hashtags': [{'text': 'teamturkey', 'indices'...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,373,0,False,False,en,NaN,NaN,"{'media': [{'id': 1418816039354314762, 'id_str...",False,NaN
3,Sat Jul 24 22:54:48 +0000 2021,1419068576884043778,1419068576884043778,RT @meongpundung: From High School Spring Tour...,False,"{'hashtags': [{'text': 'ハイキュー', 'indices': [93...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,283,0,False,False,en,NaN,NaN,NaN,NaN,NaN
4,Sat Jul 24 22:54:46 +0000 2021,1419068568835170307,1419068568835170307,RT @katieledecky: Sometimes the defining momen...,False,"{'hashtags': [{'text': 'TheVisaAward', 'indice...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,13,0,False,False,en,NaN,NaN,NaN,NaN,NaN
5,Sat Jul 24 22:54:46 +0000 2021,1419068567467745293,1419068567467745293,RT @ESPNFC: Barbra Banda has scored two hat tr...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,971,0,False,False,en,NaN,NaN,NaN,NaN,NaN
6,Sat Jul 24 22:54:46 +0000 2021,1419068567320936462,1419068567320936462,NETHERLANDS vs BRAZIL | Women's Football - Hig...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://www.footballorgin.com/"" rel=""n...",NaN,None,...,0,0,False,False,en,NaN,NaN,NaN,NaN,NaN
7,Sat Jul 24 22:54:45 +0000 2021,1419068565878099968,1419068565878099968,RT @kylesockwell: Olympic swimming is the best...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,118,0,False,False,en,NaN,NaN,NaN,NaN,NaN
8,Sat Jul 24 22:54:45 +0000 2021,1419068563197890565,1419068563197890565,The funniest olympic sport 😂😂😂 https://t.co/ke...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,0,0,False,False,en,1.418943e+18,1418942556268859392,NaN,False,{'created_at': 'Sat Jul 24 14:34:02 +0000 2021...
9,Sat Jul 24 22:54:43 +0000 2021,1419068557523050497,1419068557523050497,@PolitelyRages What channel I haven’t seen any...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",1.419049e+18,1419048967753580556,...,0,0,False,False,en,NaN,NaN,NaN,NaN,NaN


In [6]:
# we can use pandas to put data directly into an s3 bucket
#prepare csv file name   
filename = 'lab-03/'#specify location of s3:/{my-bucket}/
groupname= 'Group_Xiaolan_' #name of your group
datetime = time.strftime("%Y%m%d") #timestamp
filenames3 = "%s%s%s.csv"%(filename,groupname,datetime) #name of the filepath and csv file
print(filenames3)


#encoding must be adjusted to accommodate abnormal characters. Use s3fs to write to S3 bucket
byte_encoded_df = df.to_csv(None).encode() #encodes file as binary
s3 = s3fs.S3FileSystem(anon=False)
with s3.open(filenames3, 'wb') as file:
    file.write(byte_encoded_df) #writes byte-encoded file to s3 location

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

lab-03/Group_Xiaolan_20210724.csv
Successfull uploaded file to location:lab-03/Group_Xiaolan_20210724.csv
